# Incidenti

In [1]:
import geopandas as gpd

from my_paths import *

Carico, elimino municipi null, converto i municipi a interi

Mantengo solo il 2023 e raggruppo per municipio tramite sum su ogni colonna

In [3]:
df_mese_municipio = gpd.pd.read_csv(PATH_INCIDENTI_RAW, sep=";")
df_mese_municipio = df_mese_municipio.rename(columns={"Municipio": "MUNICIPIO"})
df_mese_municipio = df_mese_municipio[df_mese_municipio["MUNICIPIO"].notna()]
df_mese_municipio["MUNICIPIO"] = df_mese_municipio["MUNICIPIO"].astype(int)
df_mese_municipio = df_mese_municipio[df_mese_municipio["Anno"] == 2023]
df_mese_municipio = df_mese_municipio.drop(columns=["Anno", "Mese"])
df_mese_municipio = df_mese_municipio.groupby("MUNICIPIO", as_index=False).sum().reset_index(drop=True)

Carico le geometrie dei municipi dal file raw e aggiungo la geometria sul df.  
Salvo come geojson in Clean.

In [4]:
gdf_municipi = gpd.read_file(PATH_MUNICIPI_CLEAN).to_crs(CRS_GRAD).drop(["AREA", "PERIMETRO"], axis=1)

gdf_municipi["MUNICIPIO"] = gdf_municipi["MUNICIPIO"].astype(int)
gdf_municipi = gdf_municipi.sort_values("MUNICIPIO").reset_index(drop=True)

df_mese_municipio["geometry"] = gdf_municipi["geometry"]
gpd.GeoDataFrame(df_mese_municipio, geometry="geometry", crs=CRS_GRAD).to_file(PATH_INCIDENTI_CLEAN, driver="GeoJSON")